In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import config

from IPython.display import HTML

### Numerically propagate Newtons Gravity model for many particles

In [25]:
def init_r(D, n):
    r = np.random.normal(loc = D, scale = D/4, size = n)
    thetas = np.random.rand(n)*2*np.pi
    
    rs = np.zeros((n,2))
    rs[:,0] = r*np.cos(thetas)
    rs[:,1] = r*np.sin(thetas)
    return rs

def init_v(r, v):
    
    rn = np.zeros((len(r), 3))
    rn[:,:2] = r
    
    zs = np.zeros(rn.shape)
    zs[:,2] = v
    
    return np.cross(zs,rn)[:,:2]

def init_reset(D,V, n = 100):
    r = init_r(D, n) #np.random.rand(8,2)*10 #np.array([[0.,-1.], [1.,0.], [0.,1.], [-1.,0]])
    v = init_v(r, V) 
    a = np.zeros((len(r), len(r[0])))
    F = np.zeros((len(r), len(r[0])))
    config.masses_ = np.linspace(.1, 100, len(r))*2 #np.array([10000., 1]) #
    #config.masses_[0] = 1000
    #r[0] = [0,0]
    #v[0] = [0,0]
    
    dt = .01
    nframes = 1000
    config.mask_ = np.zeros(n).astype(bool)
    config.r_ = np.ma.array(r, mask=False)
    config.v_ = np.ma.array(v, mask=False)
    config.a_ = np.ma.array(a, mask=False)
    config.F_ = np.ma.array(F, mask=False)
    config.dir_ij_arr_ = np.ma.zeros((len(r), len(r), len(r[0])))
    config.collision_threshold_ = .1
    return dt, nframes
    
dt, nframes = init_reset(3, 1)    

In [26]:
def dir_ij():
    """
    Update the direction matrix from each particle to every other particle
    """
    idxs = np.where(~config.r_.mask[:,0])[0]
    rsubst = config.r_[idxs].copy()
    for i,j in enumerate(idxs):
        config.r_.mask[j] = True
        config.dir_ij_arr_[j] = config.r_ - rsubst[i]
        config.r_.mask[j] = False
    

def mask_all():
    
    config.r_.mask[:,:] = config.mask_[:,np.newaxis]
    config.v_.mask[:,:] = config.mask_[:,np.newaxis]
    config.a_.mask[:,:] = config.mask_[:,np.newaxis]
    config.F_.mask[:,:] = config.mask_[:,np.newaxis]
    config.dir_ij_arr_.mask[:,:,:] = config.mask_[np.newaxis,:,np.newaxis]
    
def Forces():
    """
    Calculate the gravitational Force on each particle due to other particles
    
    returns
    -------
    Forces : Matrix with force on particle i on i:th row
    """
    dir_ij()
    
    norms = np.linalg.norm(config.dir_ij_arr_, axis = 2, keepdims = True) #*masses
    
    collision_idxs = np.transpose(np.where(norms < config.collision_threshold_)).astype(int)

    firstSmaller = config.masses_[collision_idxs[:,0]] < config.masses_[collision_idxs[:,1]]
    annihilate_idxs = collision_idxs[firstSmaller,0]
    grow_bigger_idxs = collision_idxs[firstSmaller,1]
    
    # Update the masses The same is required for velocities!!
    config.masses_[grow_bigger_idxs] += config.masses_[annihilate_idxs]
    config.mask_[annihilate_idxs] = True
    
    print(annihilate_idxs)
    # mask everything since particles have vanished
    mask_all()
    
    masses_arr = np.tile(config.masses_, len(config.r_)).reshape(config.dir_ij_arr_.shape[:2] + (1,))
    
    #print(config.dir_ij_arr_[1,:,:])
    
    dirs_m = config.dir_ij_arr_/norms*masses_arr
    config.F_ = np.dot(np.diag(config.masses_), (dirs_m/norms**2).sum(axis = 1, keepdims = True)[:,0,:])
    return config.F_

_ = Forces()

[ 6 34 63]


#### From Forces calculate the everything needed :) 

In [20]:
def update_positions_and_velocities(): #r, v, dt = .01):
    
    config.a_ = np.dot(np.diag(1/config.masses_), Forces())
    config.r_ += dt*config.v_ + .5*config.a_*dt**2
    config.v_ += config.a_*dt
    
#update_positions_and_velocities()    

#### Animate using pyplot

In [27]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

rnorm = 15
dt, nframes = init_reset(rnorm, .25, 200)
fig, ax = plt.subplots(figsize = (8,8))

scat = plt.scatter([], [], marker = '.', animated=True)

def data_gen():
    cnt = 0
    while cnt < nframes:
        print(cnt)
        update_positions_and_velocities()
        cnt += 1
        yield config.r_



def init():
    plt.axis('equal')
    plt.axis('off')
    ax.set_xlim(-10*rnorm,10*rnorm)
    ax.set_ylim(-10*rnorm,10*rnorm)

    return scat,

def update(data):
    update_positions_and_velocities()
    
    #print((~config.r_.mask[:,0]).sum())
    scat.set_offsets(config.r_)
    return scat,

ani = FuncAnimation(fig, update, nframes,
                    init_func=init, blit=True, 
                    interval = dt*1000)
HTML(ani.to_html5_video())
#plt.clf()

[]
[36]
[36 50 91]
[ 6 36 37 50 91]
[ 6 36 37 50 62 81 91]
[  6  36  37  38  39  50  62  81  91 123]
[  6  22  36  37  38  39  50  62  81  91 123]
[  6  22  36  37  38  39  50  62  81  91  93 123 137]
[  6  22  36  37  38  39  50  62  81  91  93 123 128 137]
[  6  22  36  37  38  39  50  62  81  91  93 123 128 137]
[  6  22  36  37  38  39  50  62  81  91  93 123 128 137]
[  6  22  36  37  38  39  50  58  62  81  91  93 118 123 128 137]
[  6  22  36  37  38  39  50  58  62  76  80  81  91  93 118 123 128 137]
[  6  22  30  36  37  38  39  50  52  58  62  76  80  81  91  93 118 123
 128 137]
[  6  22  23  30  36  37  38  39  50  52  58  62  76  80  81  91  93 118
 123 128 136 137]
[  6  22  23  30  36  37  38  39  50  52  58  62  76  80  81  91  93 118
 121 123 128 136 137]
[  6  22  23  30  36  37  38  39  50  52  58  62  76  80  81  91  93 118
 121 123 128 136 137]
[  6  22  23  29  30  36  37  38  39  50  52  58  62  76  80  81  91  93
 118 121 123 128 136 137]
[  6  22  23  29  30  

[  2   6  13  14  22  23  29  30  36  37  38  39  45  46  49  50  52  58
  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137 141
 142]
[  2   6  13  14  22  23  29  30  36  37  38  39  45  46  49  50  52  58
  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137 141
 142]
[  2   6  13  14  22  23  29  30  36  37  38  39  45  46  49  50  52  58
  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137 141
 142]
[  2   6  13  14  22  23  29  30  36  37  38  39  45  46  49  50  52  58
  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137 141
 142]
[  2   6  13  14  22  23  29  30  36  37  38  39  45  46  49  50  52  58
  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137 141
 142]
[  2   6  13  14  22  23  29  30  36  37  38  39  45  46  49  50  52  58
  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137 141
 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  6

[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49

[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49

[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49

...

[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49

[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49

[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49

[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49

[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49  50  52
  58  62  65  67  76  80  81  86  91  92  93 118 121 123 124 128 136 137
 141 142]
[  2   6  13  14  21  22  23  29  30  36  37  38  39  45  46  49